# Collecting Art Data

In [4]:
import requests
import json
from dotenv import load_dotenv
import os

In [44]:
import requests
import json
import time

# Define the API base URL and your API key
API_BASE_URL = "https://api.si.edu/openaccess/api/v1.0"
API_KEY = "npa7Ayz4G1ZQBW0EA2P3UfBjIR6Pcfh8covOkegn"  # Replace with your actual API key

def fetch_artworks(start_year, end_year, country, rows=100):
    # Initial search URL for artworks in the US between 1929 and 2024
    url = f"{API_BASE_URL}/search"
    
    params = {
        "q": f"date:[{start_year} TO {end_year}] AND place:{country}",
        "rows": rows,
        "sort": "newest",  # Sort by the latest entries
        "api_key": API_KEY
    }
    
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        return response.json().get('response', {}).get('rows', [])
    else:
        print(f"Error fetching artworks: {response.status_code}")
        return []

def fetch_additional_details(artwork_id):
    # Fetch content and terms (topic, culture) for each artwork
    content_url = f"{API_BASE_URL}/content/{artwork_id}"
    terms_url = f"{API_BASE_URL}/terms/topic"
    
    params = {"api_key": API_KEY}
    
    # Fetch the content details of the artwork
    content_response = requests.get(content_url, params=params)
    terms_response = requests.get(terms_url, params=params)
    
    if content_response.status_code == 200 and terms_response.status_code == 200:
        content_data = content_response.json()
        terms_data = terms_response.json()
        
        # Extract relevant details
        topic = terms_data.get('response', {}).get('rows', [])
        culture = content_data.get('response', {}).get('content', {}).get('culture', [])
        
        return topic, culture
    else:
        print(f"Error fetching details for {artwork_id}: {content_response.status_code}, {terms_response.status_code}")
        return [], []

def fetch_and_store_artworks():
    # Set the years and country for the query
    start_year = 1929
    end_year = 2024
    country = "United States"
    
    # Fetch artworks in the given range
    artworks = fetch_artworks(start_year, end_year, country)
    
    artworks_data = []
    
    for artwork in artworks:
        artwork_id = artwork.get("id")
        title = artwork.get("title")
        
        print(f"Processing artwork: {title} ({artwork_id})")
        
        # Fetch additional details for each artwork
        topic, culture = fetch_additional_details(artwork_id)
        
        # Save the artwork data
        artworks_data.append({
            "id": artwork_id,
            "title": title,
            "topic": topic,
            "culture": culture
        })
        
        # Be respectful of rate limits, so we add a small delay
        time.sleep(1)
    
    # Store all fetched artworks in a JSON file
    with open("artworks_1929_2024_us.json", "w") as json_file:
        json.dump(artworks_data, json_file, indent=4)
    
    print(f"Finished fetching and storing {len(artworks_data)} artworks.")

# Run the function
fetch_and_store_artworks()


Processing artwork: Xylocopa (Xylocopoides) californica diamesa (ld1-1643406729571-1643406757247-0)
Finished fetching and storing 1 artworks.


In [23]:
import os
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv(os.path.join(os.getcwd(), '..', '.env'))

# Access your API key from the environment variable
API_KEY = os.getenv('SMITHSONIAN_TOKEN')
print(f"API Key: {API_KEY}") 

API Key: npa7Ayz4G1ZQBW0EA2P3UfBjIR6Pcfh8covOkegn


In [27]:
import os
from dotenv import load_dotenv

# Load environment variables from the .env file in the parent directory
load_dotenv(os.path.join(os.getcwd(), '..', '.env'))

# Check if the API_KEY is being read correctly
API_KEY = os.getenv('SMITHSONIAN_TOKEN')

print(f"API_KEY: {API_KEY}")  # Ensure the API key is printed

if not API_KEY:
    print("Error: API key is missing.")

API_KEY: npa7Ayz4G1ZQBW0EA2P3UfBjIR6Pcfh8covOkegn


In [30]:
import requests

# Replace with your API key directly
API_KEY = 'npa7Ayz4G1ZQBW0EA2P3UfBjIR6Pcfh8covOkegn'

# Simple test API URL, now with the API key as a query parameter
api_url = f"https://api.si.edu/openaccess/api/v1.0/search?q=painting&rows=1&api_key={API_KEY}"

response = requests.get(api_url)

# Print the status code and the response
print(f"Status Code: {response.status_code}")
print("Response:", response.text)


Status Code: 200
Response: {
  "status": 200,
  "responseCode": 1,
  "response": {
    "rows": [
      {
        "id": "ld1-1646149545906-1646150896932-0",
        "title": "[Trade catalogs on Valdura paints : enamel paints, floor paints, roofing paints, asphalt paints, aluminum paints, rubber-based paints, primer paints ... ]",
        "unitCode": "SIL",
        "type": "edanmdm",
        "url": "edanmdm:siris_sil_683260",
        "content": {
          "freetext": {
            "date": [
              {
                "label": "Date",
                "content": "1945"
              },
              {
                "label": "Date",
                "content": "1945-"
              }
            ],
            "name": [
              {
                "label": "Author",
                "content": "American-Marietta Company"
              },
              {
                "label": "Author",
                "content": "Smithsonian Libraries American History Trade Literature Collection

In [51]:
import time
import requests
import json

# Define the function to fetch artworks
def fetch_artworks(start_year, end_year, country, rows=100, start=0, category=None, type_filter=None):
    base_url = "https://api.si.edu/openaccess/api/v1.0/search"
    api_key = "your_api_key_here"  # Use your actual API key

    params = {
        "q": f"date:{start_year}-{end_year} AND country:{country}",
        "start": start,
        "rows": rows,
        "api_key": api_key,
    }
    
    if category:
        params["category"] = category
    if type_filter:
        params["type"] = type_filter

    response = requests.get(base_url, params=params)
    data = response.json()
    
    if response.status_code == 200 and "response" in data:
        return data["response"]["rows"], data["response"]["rowCount"]
    else:
        print(f"Error fetching artworks: {response.status_code}")
        return [], 0

# Define the function to fetch additional details (topic and culture) for each artwork
def fetch_additional_details(artwork_id):
    url = f"https://api.si.edu/openaccess/api/v1.0/content/{artwork_id}"
    api_key = "your_api_key_here"  # Use your actual API key

    params = {
        "api_key": api_key
    }

    response = requests.get(url, params=params)
    data = response.json()
    
    if response.status_code == 200 and "response" in data:
        content = data["response"].get("content", {})
        topic = content.get("topic", [])
        culture = content.get("culture", [])
        return topic, culture
    else:
        print(f"Error fetching details for artwork {artwork_id}: {response.status_code}")
        return [], []

# Main function to fetch and store artworks
def fetch_and_store_artworks():
    start_year = 1929
    end_year = 2024
    country = "USA"  # Focus on artworks made in the United States
    category = "art_design"  # Focus on artworks from the 'art_design' category
    type_filter = "edanmdm"  # Focus on 'edanmdm' type (likely to be artwork)

    # Fetch artworks from the API
    artworks, total_count = fetch_artworks(start_year, end_year, country, rows=100, start=0, category=category, type_filter=type_filter)

    if total_count == 0:
        print("No artworks found. Check the query parameters.")
        return

    artworks_data = []
    total_pages = (total_count // 100) + 1  # Calculate the total number of pages
    
    print(f"Total artworks found: {total_count}, Total pages: {total_pages}")
    
    # Process the first batch of artworks
    for artwork in artworks:
        artwork_id = artwork.get("id")
        title = artwork.get("title")
        
        print(f"Processing artwork: {title} ({artwork_id})")
        
        # Fetch additional details for each artwork
        topic, culture = fetch_additional_details(artwork_id)
        
        # Save the artwork data
        artworks_data.append({
            "id": artwork_id,
            "title": title,
            "topic": topic,
            "culture": culture
        })
        
        # Be respectful of rate limits, so we add a small delay
        time.sleep(1)
    
    # Now, fetch additional pages if they exist
    for page in range(1, total_pages):
        print(f"Fetching page {page + 1}/{total_pages}...")
        artworks, _ = fetch_artworks(start_year, end_year, country, rows=100, start=page * 100, category=category, type_filter=type_filter)
        
        # Check if the new page contains any results
        if not artworks:
            print(f"No more artworks found on page {page + 1}. Stopping.")
            break
        
        for artwork in artworks:
            artwork_id = artwork.get("id")
            title = artwork.get("title")
            
            print(f"Processing artwork: {title} ({artwork_id})")
            
            # Fetch additional details for each artwork
            topic, culture = fetch_additional_details(artwork_id)
            
            # Save the artwork data
            artworks_data.append({
                "id": artwork_id,
                "title": title,
                "topic": topic,
                "culture": culture
            })
            
            # Be respectful of rate limits, so we add a small delay
            time.sleep(1)
    
    # Store all fetched artworks in a JSON file
    with open("artworks_1929_2024_filtered.json", "w") as json_file:
        json.dump(artworks_data, json_file, indent=4)
    
    print(f"Finished fetching and storing {len(artworks_data)} artworks.")

# Run the function
fetch_and_store_artworks()


Error fetching artworks: 403
No artworks found. Check the query parameters.


In [33]:
import requests

# Replace with your API key
API_KEY = 'npa7Ayz4G1ZQBW0EA2P3UfBjIR6Pcfh8covOkegn'

# A simpler query to just search for "art" and see if we get any results at all
api_url = f"https://api.si.edu/openaccess/api/v1.0/search?q=art&rows=10&api_key={API_KEY}"

response = requests.get(api_url)

# Check if the response is successful
if response.status_code == 200:
    data = response.json()  # Get the data as JSON
    result_count = data['response']['rowCount']
    print(f"Status Code: {response.status_code}")
    print(f"Number of Results: {result_count}")
    
    # Check if there are results
    if result_count > 0:
        # Show the first result
        print("First Result:", data['response']['rows'][0])
    else:
        print("No results found matching your query.")
else:
    print(f"Failed to fetch data. Status Code: {response.status_code}")
    print("Response:", response.text)


Status Code: 200
Number of Results: 605883
First Result: {'id': 'ld1-1646149545906-1646149931250-0', 'title': 'The art of understanding art / Irina D. Costache', 'unitCode': 'SIL', 'type': 'edanmdm', 'url': 'edanmdm:siris_sil_1020569', 'content': {'freetext': {'date': [{'label': 'Date', 'content': '2012'}], 'name': [{'label': 'Author', 'content': 'Costache, Irina Dana'}], 'notes': [{'label': 'Contents', 'content': 'Why should art matter to you?: A message to art beginners -- Introduction : What is art? -- Why is art made?: The purposes of art: a brief overview from A to Z -- Part I. Making art : Artists and patrons : Originality, authorship, authenticity; Appropriations; Attributions and studio practices; Collaborations; Artists and multiple works; Artists and artisans; Anonymous artists ; The creative process : Self reflections; The formative years: education, family life, and personal values ; Patronage : Private patronage; Religious patronage; Royal patronage; State, city, and commu

In [12]:
# Load the Smithsonian API key from .env file
load_dotenv()
SMITHSONIAN_TOKEN = os.getenv("SMITHSONIAN_TOKEN")

def fetch_artworks(start_year, end_year, place, output_file):
    base_url = "https://api.si.edu/openaccess/api/v1.0/search"
    query = f"category:\"Art\" AND date:{start_year}-{end_year} AND place:\"{place}\""
    params = {
        "q": query,
        "rows": 1000,
        "api_key": SMITHSONIAN_TOKEN
    }
    url_with_params = f"{base_url}?{urlencode(params)}"  # Define URL here

    try:
        headers = {"User-Agent": "Mozilla/5.0"}
        req = Request(url_with_params, headers=headers)
        response = urlopen(req).read()
        data = json.loads(response.decode("utf-8"))
        
        # Debugging prints
        print(f"Query URL: {url_with_params}")
        print(f"API Response: {data}")

    except Exception as e:
        print(f"Error while fetching artworks: {e}")

headers = {
    "User-Agent": "Mozilla/5.0",
    "Authorization": f"Bearer {SMITHSONIAN_TOKEN}"  # Add this header if the API requires it
}



In [14]:
import os
import json
from urllib.request import Request, urlopen
from urllib.parse import urlencode
from dotenv import load_dotenv

# Load the Smithsonian API key from .env file
load_dotenv()
SMITHSONIAN_TOKEN = os.getenv("SMITHSONIAN_TOKEN")

def fetch_terms_by_category(category, output_file, starts_with=None):
    """
    Fetch terms from the Smithsonian API for a specified category and save to JSON.

    Args:
        category (str): The term category (e.g., "culture", "place", "topic").
        output_file (str): Name of the JSON file to save the fetched data.
        starts_with (str, optional): Optional prefix filter for terms.

    Returns:
        None
    """
    base_url = f"https://api.si.edu/openaccess/api/v1.0/terms/{category}"
    
    # Query parameters
    params = {
        "api_key": SMITHSONIAN_TOKEN
    }
    if starts_with:
        params["starts_with"] = starts_with

    # Encode the parameters for the URL
    url_with_params = f"{base_url}?{urlencode(params)}"

    # Make the request
    try:
        headers = {"User-Agent": "Mozilla/5.0"}
        req = Request(url_with_params, headers=headers)
        response = urlopen(req).read()
        
        # Parse the JSON response
        data = json.loads(response.decode("utf-8"))
        
        # Save the data to a JSON file
        with open(output_file, "w") as json_file:
            json.dump(data, json_file, indent=4)

        print(f"Data successfully saved to {output_file}")
    except Exception as e:
        print(f"Error while fetching data: {e}")

# Example: Fetch terms for the category "culture" and save to a file
fetch_terms_by_category("culture", "culture_terms.json")


Data successfully saved to culture_terms.json
